Experiment with spec conversion

In [69]:
from utils.spectrogram_converter import SpectrogramConverter
from utils.spectrogram_params import SpectrogramParams
from utils.riff_util import image_util
import os
from pydub import AudioSegment
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

params = SpectrogramParams()

converter = SpectrogramConverter(params)


In [70]:
# experiment label
label = "Bird vocalization-bird call-bird song"

# set paths
wav_source = "./AudioSet/wav/" + label
spec_dest = "./AudioSet/spec/" + label
os.makedirs(spec_dest, exist_ok=True)

test = AudioSegment.from_wav(wav_source + "/EtPIEB0fbzA.wav")

spec_test = converter.spectrogram_from_audio(test)

spec_test = image_util.image_from_spectrogram(spec_test)

spec_test.save(spec_dest + "test.png")

os.system(f" start {spec_dest}")



sh: 1: start: not found


32512

In [71]:
from utils.spectrogram_image_converter import SpectrogramImageConverter

converter = SpectrogramImageConverter(params)

image = Image.open(spec_dest + "test.png")

audio_test = converter.audio_from_spectrogram_image(image)

audio_test.export(spec_dest + "test_conversion.wav", format="wav")





<_io.BufferedRandom name='./AudioSet/spec/Bird vocalization-bird call-bird songtest_conversion.wav'>